In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from dnn_classifier import DNNClassifer
from random import randint
from visualizer import *
from random import randint

2023-01-12 14:28:50.092880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-12 14:28:50.093078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-12 14:28:50.093092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [7]:
data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
fm = FeatureManager()
fm.build_feature(data=data,lags=0)

In [19]:
take_profit_rate = 0.1
stop_loss_rate = 0.05
max_duration = 7

def prepare_trade_forward_data(data):
    
    fwd_cols = []

    data["long_take_profit"] = data["Close"]*(1+take_profit_rate)
    data["long_stop_loss"] = data["Close"]*(1-stop_loss_rate)

    data["short_take_profit"] = data["Close"]*(1-take_profit_rate)
    data["short_stop_loss"] = data["Close"]*(1+stop_loss_rate)


    data["cum_return_forward_0"] = data["returns"]
    data["long_decision_forward_0"] = 0
    data["short_decision_forward_0"] = 0
    
     
    for i in range(1,max_duration+1):
        
        data["High_forward_{}".format(i)] = data["High"].shift(-i)
        data["Low_forward_{}".format(i)] = data["Low"].shift(-i)
        data["cum_return_forward_{}".format(i)] = data["cum_return_forward_{}".format(i-1)]+data["returns"].shift(-i)

        long_str = "long_decision_forward_{}".format(i)
        short_str = "short_decision_forward_{}".format(i)

        fwd_cols.append(long_str)
        fwd_cols.append(short_str)

        data["long_random"] = [randint(0,100) for i in range(0,len(data))]
        data["short_random"] = [randint(0,100) for i in range(0,len(data))]
        
        #Temporarily set all open as closed from previous timeframe
        data[long_str] = 2
        data[short_str] = 2

        # Compute future outcome if long position
        # If long position is still open and only stop-loss reached
        data.loc[
            (data["Low_forward_{}".format(i)]<data["long_stop_loss"])&
            (data["High_forward_{}".format(i)]<data["long_take_profit"])&
            (data["long_decision_forward_{}".format(i-1)]==0)
            ,long_str
            ] = -1

        # If long position is still open and only take-profit reached
        data.loc[
            (data["Low_forward_{}".format(i)]>data["long_stop_loss"])&
            (data["High_forward_{}".format(i)]>data["long_take_profit"])&
            (data["long_decision_forward_{}".format(i-1)]==0)
            ,long_str
            ] = 1
        
        # If long position is still open and both take-profit and stop-loss reached, odd say it was stoploss
        data.loc[
            (data["Low_forward_{}".format(i)]<data["long_stop_loss"])&
            (data["High_forward_{}".format(i)]>data["long_take_profit"])&
            (data["long_decision_forward_{}".format(i-1)]==0)&
            (data["long_random"]<50)
            ,long_str
            ] = -1
        
        # If long position is still open, both take-profit and stop-loss reached, odd say it was take-profit
        data.loc[
            (data["Low_forward_{}".format(i)]<data["long_stop_loss"])&
            (data["High_forward_{}".format(i)]>data["long_take_profit"])&
            (data["long_decision_forward_{}".format(i-1)]==0)&
            (data["long_random"]>=50)
            ,long_str
            ] = 1
        
        # If long position is still open, both take-profit and stop-loss not reached, leave it as open
        data.loc[
            (data["Low_forward_{}".format(i)]>data["long_stop_loss"])&
            (data["High_forward_{}".format(i)]<data["long_take_profit"])&
            (data["long_decision_forward_{}".format(i-1)]==0) 
            ,long_str
            ] = 0
    
        # Compute future outcome if short position
        # If short position is still open and only stop-loss reached
        data.loc[
            (data["High_forward_{}".format(i)]>data["short_stop_loss"])&
            (data["Low_forward_{}".format(i)]>data["short_take_profit"])&
            (data["short_decision_forward_{}".format(i-1)]==0)
            ,short_str
            ] = -1

        # If short position is still open and only take-profit reached
        data.loc[
            (data["High_forward_{}".format(i)]<data["short_stop_loss"])&
            (data["Low_forward_{}".format(i)]<data["short_take_profit"])&
            (data["short_decision_forward_{}".format(i-1)]==0)
            ,short_str
            ] = 1
        
        # If short position is still open and both take-profit and stop-loss reached, odd say it was stoploss
        data.loc[
            (data["High_forward_{}".format(i)]>data["short_stop_loss"])&
            (data["Low_forward_{}".format(i)]<data["short_take_profit"])&
            (data["short_decision_forward_{}".format(i-1)]==0)&
            (data["short_random"]<50)
            ,short_str
            ] = -1
        
        # If short position is still open, both take-profit and stop-loss reached, odd say it was take-profit
        data.loc[
            (data["High_forward_{}".format(i)]>data["short_stop_loss"])&
            (data["Low_forward_{}".format(i)]<data["short_take_profit"])&
            (data["short_decision_forward_{}".format(i-1)]==0)&
            (data["short_random"]>=50)
            ,short_str
            ] = 1
        
        # If short position is still open, both take-profit and stop-loss not reached, leave it as open
        data.loc[
            (data["High_forward_{}".format(i)]<data["short_stop_loss"])&
            (data["Low_forward_{}".format(i)]>data["short_take_profit"])&
            (data["short_decision_forward_{}".format(i-1)]==0) 
            ,short_str
            ] = 0
    return fwd_cols
        
cols = prepare_trade_forward_data(fm.df)
fm.df[cols].to_csv("../out/out.csv")

In [18]:
#results = pd.DataFrame(data = fm.df["decision_forward_3"].value_counts())
results = pd.DataFrame()
for i in range(1,max_duration+1):
    results["Long Forward {}".format(i)]= fm.df["long_decision_forward_{}".format(i)].value_counts()
    results["Short Forward {}".format(i)]= fm.df["short_decision_forward_{}".format(i)].value_counts()
results.to_csv("../out/results.csv")

In [21]:
fm.df[cols].head

<bound method NDFrame.head of                long_decision_forward_1  short_decision_forward_1  \
Open Time                                                          
1515801600000                       -1                         1   
1515888000000                        0                        -1   
1515974400000                       -1                         1   
1516060800000                       -1                        -1   
1516147200000                       -1                        -1   
...                                ...                       ...   
1672099200000                        0                         0   
1672185600000                        0                         0   
1672272000000                        0                         0   
1672358400000                        0                         0   
1672444800000                        2                         2   

               long_decision_forward_2  short_decision_forward_2  \
Open Time        

In [ ]:
# Robot will based on current status, make a trade with a stoploss and profit. 
# We scan the future data and record profit/loss. It may look into smaller timeframe for better info. 
# The traded result, together with current status will be feeded to classifier model to learn
def action():
    ''' Return what to do in next interval 
        -1: Close as stoploss reached
         0: Keep
         1: Close as take profit reached
         2: Closed
    '''
    outcome = 0
    return outcome